# Importer

In [ ]:
print("Importing libraries...")
import shutil
import random
import sys
import time
from pathlib import Path
import yaml
import numpy as np
import torch
import cv2
from ultralytics import YOLO
from IPython.display import Image, display, Markdown
import argparse
print("Libraries imported.")


In [ ]:


# ─── CONFIG ──────────────────────────────────────────────────────────────────────
NVD_ROOT            = Path('../nvd/nvd_dataset_full')  # must have images/{train,val,test}, labels/{train,val,test}
PCNVD_LABEL_DIR     = Path('PCNVD/Labels')
PCNVD_IMAGE_DIR     = Path('PCNVD/Images_BIGBOY')
OUT_ROOT            = Path('combined_yolo_dataset_full_no_aug')
CLASS_NAMES         = ['car']
MAX_BACKGROUND_RATIO = 0.01  # Maximum 20% of train images can be background-only

# ─── HELPERS ──────────────────────────────────────────────────────────────────────

def copy_split_from_nvd():
    """Copy pre-split NVD dataset to output directory, and count background-only train images."""
    nvd_background_count = 0
    nvd_train_count = 0

    for split in ['train', 'val', 'test']:
        img_dir = NVD_ROOT / 'images' / split
        lbl_dir = NVD_ROOT / 'labels' / split

        for sub in [f'images/{split}', f'labels/{split}']:
            (OUT_ROOT / sub).mkdir(parents=True, exist_ok=True)

        img_files = sorted(img_dir.glob('*.[jp][pn]g'))
        for img_path in img_files:
            stem = img_path.stem
            lbl_path = lbl_dir / f"{stem}.txt"
            dst_img = OUT_ROOT / f'images/{split}/{img_path.name}'
            dst_lbl = OUT_ROOT / f'labels/{split}/{stem}.txt'

            shutil.copy(img_path, dst_img)
            if lbl_path.exists():
                shutil.copy(lbl_path, dst_lbl)
                is_empty = lbl_path.stat().st_size == 0
            else:
                dst_lbl.write_text("")
                is_empty = True

            if split == 'train':
                nvd_train_count += 1
                if is_empty:
                    nvd_background_count += 1

    return nvd_train_count, nvd_background_count

def augment_train_with_pcnvd(current_total, current_background):
    """Augment training set with PCNVD images and labels, limiting total background ratio."""
    pcnvd_imgs = sorted([p for p in PCNVD_IMAGE_DIR.iterdir() if p.suffix.lower() in ['.jpg', '.png']])
    random.shuffle(pcnvd_imgs)

    total = current_total
    background = current_background
    added = 0

    for img_path in pcnvd_imgs:
        stem = img_path.stem
        lbl_path = PCNVD_LABEL_DIR / f"{stem}.txt"
        dst_img = OUT_ROOT / f"images/train/{img_path.name}"
        dst_lbl = OUT_ROOT / f"labels/train/{stem}.txt"

        if dst_img.exists():
            # Allow duplicates by suffix
            i = 1
            while dst_img.exists():
                dst_img = OUT_ROOT / f"images/train/{stem}_{i}{img_path.suffix}"
                dst_lbl = OUT_ROOT / f"labels/train/{stem}_{i}.txt"
                i += 1

        if lbl_path.exists() and lbl_path.stat().st_size > 0:
            shutil.copy(img_path, dst_img)
            shutil.copy(lbl_path, dst_lbl)
            total += 1
            added += 1
        else:
            # Check if adding this would exceed 20% background limit
            if (background + 1) / (total + 1) <= MAX_BACKGROUND_RATIO:
                shutil.copy(img_path, dst_img)
                dst_lbl.write_text("")
                background += 1
                total += 1
                added += 1
            # else: skip this background image

    print(f"➕ Augmented {added} PCNVD images into training set.")
    print(f"📊 Final train set: {total} images, {background} background-only ({(background / total):.2%})")

def write_data_yaml():
    data = {
        'path': str(OUT_ROOT.resolve()),
        'train': 'images/train',
        'val':   'images/val',
        'test':  'images/test',
        'names': CLASS_NAMES
    }
    with open(OUT_ROOT / 'data.yaml', 'w') as f:
        yaml.dump(data, f, sort_keys=False)

def prepare_dataset():
    if OUT_ROOT.exists():
        shutil.rmtree(OUT_ROOT)
    print("📦 Copying main dataset from NVD...")
    nvd_total, nvd_background = copy_split_from_nvd()
    print(f"📊 NVD train images: {nvd_total}, background-only: {nvd_background} ({(nvd_background / nvd_total):.2%})")
    # print("🔁 Augmenting training set with PCNVD (enforcing 1% max background)...")
    # augment_train_with_pcnvd(nvd_total, nvd_background)
    write_data_yaml()
    print(f"✅ Dataset prepared under {OUT_ROOT}")


if __name__ == "__main__":
    random.seed(42)
    prepare_dataset()


# Hyperparameter Tune
* data_root_path
* out_yaml_name
* trials
* epochs
* imgsz
* batch
* freeze_backbone
* patience
* seed

In [ ]:


def tune_hyperparameters(
    data_root_path,
    out_yaml_name,
    trials=20,
    epochs=50,
    imgsz=640,
    batch=16,
    freeze_backbone=False,
    patience=10,
    seed=42,
):
    """
    Run hyperparameter tuning (YOLO .tune) against a dataset and save the best hyp yaml.
    """

    # ---- Seeding for reproducibility ----
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    data_root = Path(data_root_path)
    if not data_root.exists():
        raise FileNotFoundError(f"Dataset root not found: {data_root}")

    OUT_ROOT.mkdir(parents=True, exist_ok=True)

    # write a temporary data.yaml pointing to the provided dataset root
    temp_data_yaml = OUT_ROOT / f"data_for_tune_{int(time.time())}.yaml"
    data = {
        "path": str(data_root.resolve()),
        "train": "images/train",
        "val": "images/val",
        "test": "images/test",
        "names": CLASS_NAMES,
    }
    with open(temp_data_yaml, "w") as f:
        yaml.dump(data, f, sort_keys=False)

    # run tuning
    model = YOLO("yolov8m.pt")
    tune_name = f"hp_tune_{temp_data_yaml.stem}"

    # map bool -> freeze layers (adjust number if you want deeper freeze)
    freeze = 10 if freeze_backbone else 0

    model.tune(
        data=str(temp_data_yaml),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=tune_name,
        freeze=freeze_backbone,
        optimizer='AdamW',
        augment=True,
        patience=patience,
        iterations=trials,
    )

    # find the most recent hyperparameter file produced under runs/
    candidates = []
    # Newer Ultralytics: best_hyperparameters.yaml
    candidates.extend(Path("runs").rglob("best_hyperparameters.yaml"))
    # Older style (or custom): hyp.yaml
    candidates.extend(Path("runs").rglob("hyp.yaml"))

    if not candidates:
        print(
            "No best_hyperparameters.yaml or hyp.yaml found under runs/. "
            "Tuning may have failed or the output filename changed."
        )
        try:
            temp_data_yaml.unlink()
        except Exception:
            pass
        return None

    candidates.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    best_hyp = candidates[0]

    dest = OUT_ROOT / out_yaml_name
    shutil.copy(best_hyp, dest)
    print(f"Tuned hyperparameters copied to: {dest}")

    # clean up temporary data yaml
    try:
        temp_data_yaml.unlink()
    except Exception:
        pass

    return dest


In [ ]:

# Defaults (can still be overridden via CLI)
OUT_ROOT    = Path('combined_yolo_dataset_full_no_aug')
EXPERIMENT  = 'yolov8m_big_NVD_augment_less_background'
EPOCHS      = 200
IMGSZ       = 640
BATCH       = 16
FREEZE_BACKBONE = False
PATIENCE    = 10
CONF_THRESH = 0.001
IOU_THRESH  = 0.50


def train(
    data_root: Path,
    experiment_name: str,
    hyp_path: Path | None = None,
    epochs: int = EPOCHS,
    imgsz: int = IMGSZ,
    batch: int = BATCH,
    freeze_backbone: bool = FREEZE_BACKBONE,
    patience: int = PATIENCE,
):
    """Train YOLOv8 on a given dataset + optional hyp.yaml."""
    model = YOLO('yolov8m.pt')

    # If no hyp_path provided, fall back to your manual dict
    if hyp_path is None:
        hyp = {
            'lr0': 0.003,
            'lrf': 0.1,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            'box': 0.5,
            'cls': 0.05,
            'dfl': 1.5,
            'label_smoothing': 0.0,
            'nbs': 64,
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 5.0,
            'translate': 0.1,
            'scale': 0.4,
            'shear': 2.0,
            'perspective': 0.001,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.2,
            'copy_paste': 0.1,
        }

        hyp_path = data_root / 'hyp.yaml'
        hyp_path.parent.mkdir(parents=True, exist_ok=True)
        with open(hyp_path, 'w') as f:
            yaml.dump(hyp, f)

    # Train
    model.train(
        data=str(data_root / "data.yaml"),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=experiment_name,   # ← training name!
        optimizer="AdamW",
        freeze=FREEZE_BACKBONE,
        augment=True,
        patience=PATIENCE,
        cfg=str(hyp_path),
    )

    # Path to YOLO’s saved best model
    best_weight_src = (
        Path("runs") / "train" / experiment_name / "weights" / "best.pt"
    )

    # Path where YOU want the best model saved
    best_weight_dst = Path("models") / f"{experiment_name}_best.pt"
    best_weight_dst.parent.mkdir(exist_ok=True)

    if not best_weight_src.exists():
        raise FileNotFoundError(f"Trained weights not found: {best_weight_src}")

    try:
        shutil.copy(best_weight_src, best_weight_dst)
    except Exception as e:
        raise RuntimeError(f"Failed to copy best weights from {best_weight_src} to {best_weight_dst}: {e}") from e

    print(f"\nBest model saved as: {best_weight_dst}")


def infer(
    data_root: Path,
    experiment_name: str,
    imgsz: int = IMGSZ,
    conf: float = CONF_THRESH,
    iou: float = IOU_THRESH,
):
    """Run inference using the trained weights from a given experiment."""
    weights = Path("models") / f"{experiment_name}_best.pt"
    if not weights.exists():
        sys.exit(f"ERROR: model not found: {weights}")

    model = YOLO(str(weights))


    model = YOLO(str(weights))
    results = model.predict(
        source=str(data_root / 'images' / 'val'),
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        save=True,
    )

    print("\nInference Summary:")
    for r in results:
        confs = r.boxes.conf.cpu().numpy() if r.boxes.conf.nelement() else []
        print(f"{r.orig_shape} → {len(confs)} detections, scores: {confs}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train and infer YOLOv8 on a dataset.")
    parser.add_argument('--data_root', type=Path, default=OUT_ROOT, help='Path to dataset root.')
    parser.add_argument('--experiment', type=str, default=EXPERIMENT, help='Experiment name for outputs.')
    parser.add_argument('--hyp_path', type=Path, default=None, help='Path to hyperparameter YAML file.')
    parser.add_argument('--epochs', type=int, default=EPOCHS, help='Number of training epochs.')
    parser.add_argument('--imgsz', type=int, default=IMGSZ, help='Image size for training and inference.')
    parser.add_argument('--batch', type=int, default=BATCH, help='Batch size for training.')
    parser.add_argument('--freeze_backbone', action='store_true', help='Whether to freeze backbone during training.')
    parser.add_argument('--patience', type=int, default=PATIENCE, help='Early stopping patience.')
    parser.add_argument('--conf_thresh', type=float, default=CONF_THRESH, help='Confidence threshold for inference.')
    parser.add_argument('--iou_thresh', type=float, default=IOU_THRESH, help='IOU threshold for inference.')

    args = parser.parse_args()

    print("🚀 Starting training...")
    train(
        data_root=args.data_root,
        experiment_name=args.experiment,
        hyp_path=args.hyp_path,
        epochs=args.epochs,
        imgsz=args.imgsz,
        batch=args.batch,
        freeze_backbone=args.freeze_backbone,
        patience=args.patience,
    )

    print("🔍 Starting inference...")
    infer(
        data_root=args.data_root,
        experiment_name=args.experiment,
        imgsz=args.imgsz,
        conf=args.conf_thresh,
        iou=args.iou_thresh,
    )

In [ ]:
import random
from pathlib import Path

import cv2
import numpy as np
from ultralytics import YOLO
from IPython.display import display, Image, Markdown


# ─── CONFIG ─────────────────────────────────────────────────────────────────────
WEIGHTS       = "runs/detect/yolov8m_big_NVD_augment_less_background_best/weights/best.pt"
DATA_YAML     = "combined_yolo_dataset_full/data.yaml"

IMG_ROOT      = Path("../nvd/nvd_dataset_full/images/test")
LABEL_DIR     = Path("../nvd/nvd_dataset_full/labels/test")
OUTPUT_DIR    = Path("annotated_tests")

CONF_THRESH   = 0.5
IOU_THRESH    = 0.5   # currently unused, but kept for future TP/FP analysis

CLASS_ID      = 0     # class index, e.g. 'car'
NUM_VIS       = 30    # how many images to visualize
NUM_EVAL_RUNS = 5     # how many times to run model.val() and average metrics

RANDOM_SEED   = 15
# ────────────────────────────────────────────────────────────────────────────────


random.seed(RANDOM_SEED)


# ─── HELPERS ────────────────────────────────────────────────────────────────────
def get_gt_boxes(label_file: Path, class_id: int):
    """Read YOLO-format GT boxes for a specific class from a label file."""
    boxes = []
    if not label_file.exists():
        return boxes
    with open(label_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            if int(parts[0]) == class_id:
                _, x_center, y_center, width, height = map(float, parts)
                boxes.append((x_center, y_center, width, height))
    return boxes


def yolo_to_xyxy(box, img_w: int, img_h: int):
    """Convert normalized YOLO box (xc, yc, w, h) to pixel [x1, y1, x2, y2]."""
    x_c, y_c, w, h = box
    x1 = int((x_c - w / 2) * img_w)
    y1 = int((y_c - h / 2) * img_h)
    x2 = int((x_c + w / 2) * img_w)
    y2 = int((y_c + h / 2) * img_h)
    return [x1, y1, x2, y2]


def find_valid_images(img_root: Path, label_dir: Path, class_id: int):
    """
    Return all images that have at least one GT box of CLASS_ID,
    and a cache of GT boxes keyed by image path.
    """
    valid_images = []
    gt_cache = {}

    for img_path in sorted(img_root.glob("*.*")):
        if img_path.suffix.lower() not in (".jpg", ".jpeg", ".png"):
            continue
        lbl_path = label_dir / f"{img_path.stem}.txt"
        gt_boxes = get_gt_boxes(lbl_path, class_id)
        if gt_boxes:
            valid_images.append(img_path)
            gt_cache[img_path] = gt_boxes

    return valid_images, gt_cache


def evaluate_multiple_runs(model, data_yaml: str, class_id: int,
                           conf_thresh: float, num_runs: int):
    """
    Run model.val() num_runs times and return:
      - avg metrics (dict)
      - std metrics (dict)
      - list of metrics per run
    """
    metrics_list = []

    for i in range(num_runs):
        print(f"\n▶ Evaluation run {i + 1}/{num_runs} ...")
        results = model.val(
            data=data_yaml,
            conf=conf_thresh,
            split="test",
            verbose=False,
        )
        m = results.box

        precision, recall, ap50, ap = m.class_result(class_id)
        metrics = {
            "precision": precision,
            "recall":    recall,
            "ap50":      ap50,
            "ap":        ap,
            "map50":     m.map50,
            "map":       m.map,
        }
        metrics_list.append(metrics)

    # average and std over runs
    keys = metrics_list[0].keys()
    avg = {k: float(np.mean([m[k] for m in metrics_list])) for k in keys}
    std = {k: float(np.std([m[k] for m in metrics_list])) for k in keys}

    print(f"\n🔍 Averaged Evaluation Metrics over {num_runs} runs:")
    print(f"📈 Precision       : {avg['precision']:.4f} ± {std['precision']:.4f}")
    print(f"📉 Recall          : {avg['recall']:.4f} ± {std['recall']:.4f}")
    print(f"🏁 AP@0.5          : {avg['ap50']:.4f} ± {std['ap50']:.4f}")
    print(f"📊 AP@0.5:0.95     : {avg['ap']:.4f} ± {std['ap']:.4f}")
    print(f"🎯 mAP@0.5 (mean)  : {avg['map50']:.4f} ± {std['map50']:.4f}")
    print(f"🎯 mAP@0.5:0.95    : {avg['map']:.4f} ± {std['map']:.4f}")

    return avg, std, metrics_list


def visualize_samples(model,
                      gt_cache: dict,
                      class_id: int,
                      num_vis: int,
                      conf_thresh: float,
                      output_dir: Path):
    """Select images, run batched prediction, and save/show annotated samples."""
    output_dir.mkdir(parents=True, exist_ok=True)

    valid_images = list(gt_cache.keys())
    if len(valid_images) == 0:
        print("No valid images with GT for this class.")
        return

    if len(valid_images) < num_vis:
        num_vis = len(valid_images)

    selected_images = random.sample(valid_images, num_vis)

    # Batch prediction for speed
    preds = model.predict(
        source=[str(p) for p in selected_images],
        conf=conf_thresh,
        save=False,
        verbose=False,
    )

    for idx, (img_path, res) in enumerate(zip(selected_images, preds), 1):
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Warning: could not read image {img_path}")
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # Ground truth boxes (cached)
        gt_boxes_yolo = gt_cache[img_path]
        gt_boxes = [yolo_to_xyxy(box, w, h) for box in gt_boxes_yolo]

        # Draw predictions (green) for this class
        for box, cls_id, conf in zip(res.boxes.xyxy, res.boxes.cls, res.boxes.conf):
            if int(cls_id) != class_id:
                continue
            x1, y1, x2, y2 = map(int, box.cpu().numpy())
            label = f"{model.names[class_id]} {conf:.2f}"
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            ts = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            cv2.rectangle(img_rgb, (x1, y1 - ts[1] - 4),
                          (x1 + ts[0], y1), (0, 255, 0), -1)
            cv2.putText(img_rgb, label, (x1, y1 - 2),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        # Draw GT boxes (blue)
        for gb in gt_boxes:
            x1, y1, x2, y2 = gb
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img_rgb, "GT", (x1, y1 - 6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        out_path = output_dir / f"sample_{idx:02d}_{img_path.name}"
        cv2.imwrite(str(out_path), cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

        display(Markdown(f"### {idx}. {img_path.name}"))
        display(Image(filename=str(out_path)))


# ─── MAIN FLOW ──────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # 1) Load model once
    model = YOLO(WEIGHTS)

    # 2) Run evaluation multiple times and average the metrics
    avg_metrics, std_metrics, all_runs = evaluate_multiple_runs(
        model=model,
        data_yaml=DATA_YAML,
        class_id=CLASS_ID,
        conf_thresh=CONF_THRESH,
        num_runs=NUM_EVAL_RUNS,
    )

    # 3) Find valid images (with GT for CLASS_ID) and cache GT boxes
    valid_images, gt_cache = find_valid_images(IMG_ROOT, LABEL_DIR, CLASS_ID)
    print(f"\nFound {len(valid_images)} images with GT for class {CLASS_ID}.")

    # 4) Visualize NUM_VIS sample images
    visualize_samples(
        model=model,
        gt_cache=gt_cache,
        class_id=CLASS_ID,
        num_vis=NUM_VIS,
        conf_thresh=CONF_THRESH,
        output_dir=OUTPUT_DIR,
    )
